In [1]:
import re
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential,Model
from keras.layers import Dense,Bidirectional,Dropout,LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
from unidecode import unidecode
from keras.regularizers import l2
import tensorflow as tf
from keras.models import load_model
import unicodedata
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.stem.snowball import SnowballStemmer
from tensorflow.keras.preprocessing.text import Tokenizer

# Download necessary data for Portuguese
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

glove_file = 'glove.6B.50d.txt'


2023-07-31 14:39:12.846641: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 14:39:13.218974: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-31 14:39:13.219062: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-31 14:39:15.144191: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
def load_glove_embeddings(path):
    embeddings_index = {}
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

def preprocess_texts(text_list):
    stopwords_pt = set(stopwords.words('portuguese'))
    stemmer = SnowballStemmer("portuguese")

    preprocessed_texts = []

    for text in text_list:
        tokens = word_tokenize(text, language='portuguese')
        filtered_tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stopwords_pt]
        stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
        preprocessed_texts.append(stemmed_tokens)

    return preprocessed_texts

def encoding(sentences, Glove):
    Encoded_vec = []
    for sentence in sentences:
        sent_vec = []
        for token in sentence:
            word = tokenizer.index_word.get(token, '')
            if word in Glove:
                sent_vec.append(Glove[word])
            else:
                sent_vec.append(np.zeros(50))
        Encoded_vec.append(sent_vec)
    return Encoded_vec

class BiLSTMModel:
    def load_weights(self, path):
        self.model.load_weights(path)
        
    def __init__(self):
        self.model = Sequential()
        #self.model.add(Bidirectional(LSTM(200, input_shape=(None, 200), kernel_regularizer=l2(0.05))))
        #self.model.add(Dropout(0.5))
        #self.model.add(Dense(7, activation='softmax'))
        self.model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(100, 50)))
        self.model.add(Dropout(0.5))
        self.model.add(Bidirectional(LSTM(32)))
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dense(7, activation='softmax'))
        self.model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    def fit(self, X, Y, epochs, batch_size, initial_epoch=0):

        self.model.fit(X, Y, epochs=epochs, batch_size=batch_size, initial_epoch=initial_epoch)

    def evaluate(self, X, Y, batch_size):
        return self.model.evaluate(X, Y, batch_size=batch_size)

    def predict(self, X):
        return self.model.predict(X)

In [3]:

# Load GloVe word embeddings
Glove = load_glove_embeddings(glove_file)

df=pd.read_csv('ISEAR_only_text_pt_data_augmentation.csv', sep=";")
df.head()

Sentences = df['text']
Sentiments = df['sentiment']

preprocessed_texts = preprocess_texts(Sentences)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_texts)
sequences_tokens = tokenizer.texts_to_sequences(preprocessed_texts)
pad_sequences = pad_sequences(sequences_tokens, maxlen=100)
Encoded_vec = encoding(pad_sequences, Glove)
X = np.array(Encoded_vec)

enc = OneHotEncoder(handle_unknown='ignore')
Y = enc.fit_transform(np.array(Sentiments).reshape(-1,1)).toarray()

# Split the data into training and evaluation sets (90% training, 10% evaluation)
train_features, eval_features, train_labels, eval_labels = train_test_split(
    X,Y, test_size=0.1
)
# Split the training set further into training and validation sets (80% training, 20% validation)
train_features, val_features, train_labels, val_labels = train_test_split(
    train_features, train_labels, test_size=0.2
)

In [4]:

# fit the model on the input and target data
model = BiLSTMModel()
model.model.fit(train_features, train_labels, epochs=25, batch_size=64, validation_data=(val_features, val_labels))
#model.model.save('model_training25.h5')

2023-07-31 14:40:46.616723: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-31 14:40:46.616759: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-31 14:40:46.616779: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-0GDMKU0D.Home): /proc/driver/nvidia/version does not exist
2023-07-31 14:40:46.626241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(10815, 100, 50)
(10815, 7)
Epoch 1/25


2023-07-31 14:40:48.214183: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 216300000 exceeds 10% of free system memory.


169/169 [==============================] - 43s 202ms/step - loss: 1.8476 - accuracy: 0.2598 - val_loss: 1.7499 - val_accuracy: 0.3140
Epoch 2/25
169/169 [==============================] - 34s 202ms/step - loss: 1.6759 - accuracy: 0.3559 - val_loss: 1.6541 - val_accuracy: 0.3643
Epoch 3/25
169/169 [==============================] - 35s 208ms/step - loss: 1.5815 - accuracy: 0.3993 - val_loss: 1.5914 - val_accuracy: 0.3894
Epoch 4/25
169/169 [==============================] - 37s 218ms/step - loss: 1.5064 - accuracy: 0.4323 - val_loss: 1.5602 - val_accuracy: 0.4109
Epoch 5/25
169/169 [==============================] - 36s 213ms/step - loss: 1.4487 - accuracy: 0.4583 - val_loss: 1.5368 - val_accuracy: 0.4201
Epoch 6/25
169/169 [==============================] - 41s 241ms/step - loss: 1.4006 - accuracy: 0.4784 - val_loss: 1.5299 - val_accuracy: 0.4338
Epoch 7/25
133/169 [======================>.......] - ETA: 7s - loss: 1.3531 - accuracy: 0.4880

KeyboardInterrupt: 

In [ ]:

#model = BiLSTMModel()
model = load_model('model_training25.h5')
# Continue training from epoch 51 to 100
model.fit(train_features, train_labels, epochs=50, initial_epoch=25, batch_size=64, validation_data=(val_features, val_labels))
model.save('model_training50.h5')

In [ ]:

Loss, acc = model.evaluate(eval_features, eval_labels, batch_size=32)
print("Loss: %.2f" % (Loss))
print("acc: %.2f" % (acc)) 